[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io/)

# Machine Learning Methods

## UnSupervised Learning - Dimensionality Reduction - Multidimensional Scaling (MDS)

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 23/02/2023 | Royi Avital | First version                                                      |
|         |            |             |                                                                    |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/MachineLearningMethods/2023_01/0038DimensionalityReductionPCA.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.datasets import make_s_curve
from sklearn.manifold import MDS

# Miscellaneous
import os
import math
from platform import python_version
import random

# Typing
from typing import Callable, List, Tuple, Union

# Visualization
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D 
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

In [ ]:
# Configuration
%matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2


In [ ]:
# Fixel Algorithms Packages


## Dimensionality Reduction by MDS

The MDS is a non linear transformation from $\mathbb{R}^{D} \to \mathbb{R}^{d}$ where $d \ll D$.  
Given a set $\mathcal{X} = {\left\{ \boldsymbol{x}_{i} \in \mathbb{R}^{D} \right\}}_{i = 1}^{n}$ is builds the set $\mathcal{Z} = {\left\{ \boldsymbol{z}_{i} \in \mathbb{R}^{d} \right\}}_{i = 1}^{n}$ such that the distance matrices of each set are similar.

In this notebook:

 - We'll also implement the classic MDS.
 - We'll use the data set to show the effects of dimensionality reduction.  

In [ ]:
# Parameters

# Data
numSamples = 500

# Model


In [ ]:
# Auxiliary Functions

def PlotScatterData(mX: np.ndarray, vL: np.ndarray, hA:plt.Axes = None, figSize: Tuple[int, int] = FIG_SIZE_DEF, markerSize: int = MARKER_SIZE_DEF, lineWidth: int = LINE_WIDTH_DEF, axisTitle: str = None):

    if hA is None:
        hF, hA = plt.subplots(figsize = figSize)
    else:
        hF = hA.get_figure()
    
    vU = np.unique(vL)
    numClusters = len(vU)

    for ii in range(numClusters):
        vIdx = vL == vU[ii]
        hA.scatter(mX[vIdx, 0], mX[vIdx, 1], s = ELM_SIZE_DEF, edgecolor = EDGE_COLOR, label = ii)
    
    hA.set_xlabel('${{x}}_{{1}}$')
    hA.set_ylabel('${{x}}_{{2}}$')
    if axisTitle is not None:
        hA.set_title(axisTitle)
    hA.grid()
    hA.legend()

    # return hF

def PlotScatterData3D(mX: np.ndarray, vL: np.ndarray = None, vC: np.ndarray = None, hA: plt.Axes = None, figSize: Tuple[int, int] = FIG_SIZE_DEF, markerSize: int = MARKER_SIZE_DEF, lineWidth: int = LINE_WIDTH_DEF, axisTitle: str = None):

    if hA is None:
        hF, hA = plt.subplots(figsize = figSize, subplot_kw = {'projection': '3d'})
    else:
        hF = hA.get_figure()
    
    if vL is not None:
        vU = np.unique(vL)
        numClusters = len(vU)
    else:
        vL = np.zeros(mX.shape[0])
        vU = np.zeros(1)
        numClusters = 1

    for ii in range(numClusters):
        vIdx = vL == vU[ii]
        hA.scatter(mX[vIdx, 0], mX[vIdx, 1], mX[vIdx, 2], s = ELM_SIZE_DEF, c = vC, alpha = 1, edgecolor = EDGE_COLOR, label = ii)
    
    hA.set_xlabel('${{x}}_{{1}}$')
    hA.set_ylabel('${{x}}_{{2}}$')
    hA.set_zlabel('${{x}}_{{3}}$')
    if axisTitle is not None:
        hA.set_title(axisTitle)
    hA.grid()
    hA.legend()

    return hA


## Generate / Load Data

In this notebook we'll use [SciKit Learn's `make_s_curve`](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_s_curve.html) to generated data.  

In [ ]:
# Loading / Generating Data

mX, vC = make_s_curve(numSamples)


print(f'The features data shape: {mX.shape}')
print(f'The features data type: {mX.dtype}')

* <font color='red'>(**?**)</font> Do we need to scale the data?

### Plot the Data

In [ ]:
# Plot the Data

hA = PlotScatterData3D(mX, vC = vC, axisTitle = 'The S Surface')
hA.set_xlim([-2, 2])
hA.set_ylim([-2, 2])
hA.set_zlim([-2, 2])

## Applying Dimensionality Reduction - MDS

In this section we'll implement the Classic MDs:

### Non Metric (Classic) MDS

$$\min_{\left\{ \boldsymbol{z}_{i}\in\mathbb{R}^{d}\right\} }\sum_{i=1}^{N}\sum_{j=1}^{N}\left(\boldsymbol{K}_{x}\left[i,j\right]-\left\langle \boldsymbol{z}_{i},\boldsymbol{z}_{j}\right\rangle \right)^{2}$$
1. **set** $\boldsymbol{K}_{x}=-\frac{1}{2}\boldsymbol{J}\boldsymbol{D}_{x}\boldsymbol{J}$  
where $\boldsymbol{J}=\left(\boldsymbol{I}-\frac{1}{N}\boldsymbol{1}\boldsymbol{1}^{T}\right)$
2. Decompose $\boldsymbol{K}_{x}=\boldsymbol{W}\boldsymbol{\Lambda}\boldsymbol{W}$
3. **set** $\boldsymbol{Z}=\boldsymbol{\Lambda}_{d}^{\frac{1}{2}}\boldsymbol{W}_{d}^{T}$

* <font color='brown'>(**#**)</font> The non metric MDS matches (Kernel) PCA.
* <font color='brown'>(**#**)</font> It is assumed above that the eigen values matrix $\boldsymbol{\Lambda}$ is sorted.
* <font color='brown'>(**#**)</font> For Euclidean distance there is a closed form solution (As with teh PCA).



In [ ]:
# Classic MDS Implementation

def ClassicalMDS(mD, lowDim):
    numSamples = mD.shape[0]

    mJ     = np.eye(numSamples) - ((1 / numSamples) * np.ones((numSamples, numSamples)))
    mK     = -0.5 * mJ @ mD @ mJ #<! Due to the form of mJ one can avoid teh matrix multiplication
    vL, mW = np.linalg.eigh(mK)
    
    # Sort Eigen Values
    vIdx   = np.argsort(-vL)
    vL     = vL[vIdx]
    mW     = mW[:, vIdx]
    # Reconstruct
    mZ     = mW[:, :lowDim] * np.sqrt(vL[:lowDim])
    
    return mZ

In [ ]:
# Apply the MDS

# Build the Distance Matrix
mD  = sp.spatial.distance.squareform(sp.spatial.distance.pdist(mX))
# The MDS output
mZ1 = ClassicalMDS(np.squared(mD), 2)

### Plot the Mean Image  

The PCA works on a centered data.  
Hence the mean image is kept a side for the reconstruction.

In [ ]:
# Plot the Mean Image

PlotMnistImages(np.atleast_2d(oPCA.mean_), np.array(['Mean Image']), 1, 1, imgSize = tImgSize)

In [ ]:
# Plot the PCA Spectrum

vλ = oPCA.explained_variance_ratio_

hF, hA = plt.subplots(figsize = (12, 6))
hA.stem(np.sqrt(vλ[:200]), markerfmt = 'b.', label = '$\\sqrt{\lambda_i}$')
hA.set_title('Eigen Values')
hA.set_xlabel('$i$')
hA.legend()

plt.show()

In [ ]:
# Plot the Energy Ratio

vλ = oPCA.explained_variance_ratio_

hF, hA = plt.subplots(figsize = (12, 6))
hA.stem(vλ, markerfmt = 'b.', label = '$Ratio$')
hA.set_title('Variance Ratio')
hA.set_xlabel('$Component Index$')
hA.legend()

plt.show()

* <font color='brown'>(**#**)</font> Look at the rate the accumulated explained energy is accumulated.

In [ ]:
# Plot the Components

mU = oPCA.components_ #<! mU.shape = (n_components, n_features)

hF, hAs = plt.subplots(nrows = 2, ncols = 5, figsize = (12, 6))
vIdx    = list(range(5)) + list(range(numComp - 5, numComp))
for kk, hA in zip(range(10), hAs.flat):
    idx = vIdx[kk]
    mI  = np.reshape(mU[idx], tImgSize)
    hA.imshow(mI)
    hA.set_title(f'{hOrdinalNum(idx + 1)} Principal Component')
    
hF.tight_layout()
plt.show()

## PCA Reconstruction

* Encode:
$$\boldsymbol{z}_{i}=\boldsymbol{U}_{d}^{T}\left(\boldsymbol{x}_{i}-\boldsymbol{\mu}_{x}\right)$$  

* Decode:
$$\hat{\boldsymbol{x}}_{i}=\boldsymbol{U}_{d}\boldsymbol{z}_{i}+\boldsymbol{\mu}_{x}$$

In [ ]:
# Interactive Visualization 

hPlotPcaReconstruction = lambda dataIdx, numComponents: PlotPcaReconstruction(mX, dataIdx, mU, oPCA.mean_, numComponents, tImgSize, figSize = (14, 4))
dataIdxSlider = IntSlider(min = 0, max = numSamples - 1, step = 1, value = 0, layout = Layout(width = '30%'))
numComponentsSlider = IntSlider(min = 0, max = numComp, step = 1, value = 0, layout = Layout(width = '30%'))

interact(hPlotPcaReconstruction, dataIdx = dataIdxSlider, numComponents = numComponentsSlider)

* <font color='red'>(**?**)</font> Describe how the actual recognition of a given face is done.

<!-- Given the data base, each image in the data base has its own finger print on the data base: $\boldsymbol{z}_{i}$.  
Then, for a new image:

1. Calculate its encoding using the components: $\boldsymbol{z}_{new} = \boldsymbol{U}_{d}^{T} \left( \boldsymbol{x}_{new} - \boldsymbol{\mu}_{x} \right)$.
2. Calculate the distance to the closest existing finger printing: $j = \arg \min_{i} {d}_{i} = \left\| \boldsymbol{z}_{new} - \boldsymbol{z}_{i}$.
3. If ${d}_{j} \leq {\vareps}_{1}$ for a given threshold ${\vareps}_{1}$ then the face is recognized as the $j$ -th face in the data base.
4. If ${\vareps}_{1} < {d}_{j} \leq {\vareps}_{2}$ for a given threshold ${\vareps}_{2}$ then the image is not in the data base yet can be added.
5. If ${\vareps}_{2} < {d}_{j}$ for a given threshold ${\vareps}_{2}$ then the image is not considered a face image. -->